
# Reporte de Entrenamiento: Detección de Data Leakage
> - **Fecha:** 02 de Noviembre, 2025
> - **Universidad:** ICESI
> - **Curso:** Inteligencia Artificial I
> - **Estado:** 🚨 **PROBLEMA DETECTADO - CORRECCIÓN PENDIENTE**

---

## 🎯 Resumen Ejecutivo

Este reporte documenta el proceso de entrenamiento de un modelo para la clasificación de actividades y la **detección crítica de *data leakage***. El problema ha sido identificado y diagnosticado completamente a través de un análisis forense. Se ha diseñado un protocolo de corrección, pero este **aún no ha sido ejecutado**.

#### Estado Actual:
-   **✅ Problema Detectado:** *Data leakage* severo confirmado.
-   **✅ Análisis Forense:** Completado con evidencia clara.
-   **✅ Plan de Corrección:** Diseñado y documentado.
-   **🔄 Corrección y Reentrenamiento:** **PENDIENTE DE EJECUCIÓN**.

---

## 📊 Resultados Iniciales (Problemáticos)

Los resultados obtenidos en la primera fase de entrenamiento mostraron precisiones artificialmente perfectas, levantando sospechas inmediatas.

### Rendimiento con *Data Leakage*

| Algoritmo | Accuracy (Validación) | Estado |
| :--- | :--- | :--- |
| **MLP** | **1.000** | 🚨 SOSPECHOSO |
| **Random Forest** | **0.998** | 🚨 SOSPECHOSO |
| **Gradient Boosting**| **0.994** | 🚨 SOSPECHOSO |
| **SVM** | **0.991** | 🚨 SOSPECHOSO |

### Matriz de Confusión del MLP (Contaminada)

```
                 Predicción  
Real    CH   CR   GI   PP   SE
CH     370    0    0    0    0   <-- Perfección artificial
CR       0  355    0    0    0   <-- Perfección artificial  
GI       0    0  342    0    0   <-- Perfección artificial
PP       0    0    0  334    0   <-- Perfección artificial
SE       0    0    0    0  310   <-- Perfección artificial
```
> **Interpretación:** Una matriz de confusión con una diagonal perfecta (cero errores) es una evidencia contundente de *data leakage*.

---

## 🕵️ Análisis Forense: Evidencia del Problema

Se ejecutaron varias pruebas para confirmar la causa de los resultados anómalos.

#### Evidencia 1: Duplicados Entre Conjuntos de Datos
Se verificó la existencia de muestras idénticas entre los conjuntos de entrenamiento, validación y prueba.
```python
# RESULTADO OBTENIDO:
train_val_overlap = len(set(X_train_str) & set(X_val_str))     # > 0 ❌
train_test_overlap = len(set(X_train_str) & set(X_test_str))   # > 0 ❌
```
> **Conclusión:** Se confirmó la presencia de datos duplicados en todos los conjuntos, validando el *data leakage*.

#### Evidencia 2: Modelo Simple con Rendimiento Perfecto
Un modelo intencionalmente simple también alcanzó una precisión anómalamente alta.
```python
# RESULTADO OBTENIDO:
mlp_simple = MLPClassifier(hidden_layer_sizes=(50,), alpha=0.01)
simple_accuracy = 0.98+ # También sospechosamente alto
```
> **Conclusión:** El problema no reside en la complejidad del modelo, sino en la contaminación de los datos.

### Veredicto del Análisis
-   **Causa Identificada:** El proceso de *data augmentation* creó muestras sintéticas "casi-duplicadas" que fueron mezcladas incorrectamente durante la creación de los splits.
-   **Estado:** **PROBLEMA DIAGNOSTICADO - CORRECCIÓN REQUERIDA**.

---

## 🛠️ Plan de Corrección (Diseñado - No Ejecutado)

Se ha diseñado un protocolo claro para remediar el problema.

#### Paso 1: Recrear los Splits de Forma Limpia
*Este código está preparado pero aún no ha sido ejecutado.*
```python
RANDOM_STATE_CLEAN = 2025
TEST_SIZE = 0.20

# Cargar el dataset completo y original
X_complete = np.load('X_complete.npy')
y_complete = np.load('y_complete.npy')

# Crear nuevos splits garantizando independencia y estratificación
X_trainval, X_test_clean, y_trainval, y_test_clean = train_test_split(
    X_complete, y_complete,
    test_size=TEST_SIZE,
    stratify=y_complete,
    random_state=RANDOM_STATE_CLEAN
)
# ...y subsecuente split para validación
```

#### Paso 2: Verificar la Independencia de los Nuevos Splits
*Esta función se ejecutará sobre los nuevos datos para confirmar la corrección.*
```python
def verify_data_independence(X_train, X_val, X_test):
    # ... (código de verificación)
    is_clean = all(overlap == 0 for overlap in overlaps.values())
    return is_clean
```

#### Paso 3: Reentrenar el Modelo con Regularización
*Este modelo se entrenará con los datos limpios.*
```python
mlp_clean = MLPClassifier(
    hidden_layer_sizes=(100, 50),
    alpha=0.01,              # Regularización L2
    early_stopping=True,     # Para evitar overfitting
    random_state=42,
    max_iter=2000
)
```

### Resultados Esperados (Post-Corrección)
-   **Accuracy en Test:** Se espera un valor realista entre **80% y 86%**.
-   **Matriz de Confusión:** Se esperan errores naturales entre clases similares (ej. `caminar_hacia` vs `caminar_regreso`).

---

## 🚨 Limitaciones y Riesgos del Modelo Actual

El modelo en su estado actual es inválido y no debe ser utilizado.

-   **Modelo Contaminado:** La precisión del 100% es artificial y no refleja la capacidad real de generalización.
-   **No Desplegable:** El modelo fallaría estrepitosamente en un entorno de producción con datos nuevos.
-   **Métricas Falsas:** Todos los reportes de rendimiento actuales son engañosos.
-   **Plan de Despliegue Pausado:** Es imposible continuar con el despliegue hasta que el modelo sea corregido y revalidado.

---

## 📋 Próximos Pasos Obligatorios

1.  **EJECUTAR** el protocolo de corrección para generar nuevos splits de datos.
2.  **VERIFICAR** que los nuevos splits son 100% independientes.
3.  **REENTRENAR** los modelos utilizando los datos limpios.
4.  **VALIDAR** el rendimiento con las nuevas métricas realistas.
5.  **ACTUALIZAR** el reporte final con los resultados corregidos.

---

## 🎓 Lecciones Aprendidas (Hasta la Fecha)

-   **Intuición Técnica:** Siempre se debe sospechar de resultados "perfectos".
-   **Análisis Forense:** Se ha desarrollado la habilidad de diagnosticar problemas complejos en el pipeline de ML.
-   **Metodología Rigurosa:** La validación de la independencia de los datos es un paso no negociable.
-   **Honestidad Académica:** Es más valioso reportar y entender un problema que presentar resultados falsos.

---

## 🏆 Conclusión Actual

El proyecto se encuentra en un punto de inflexión crítico. Se ha identificado un error metodológico grave (*data leakage*) y se ha diseñado una solución robusta.

-   **Estado:** **Problema diagnosticado, solución pendiente de implementación.**
-   **Recomendación:** **EJECUTAR la corrección antes de la entrega final.** Un modelo con un 85% de precisión real es inmensamente más valioso, tanto académica como profesionalmente, que un modelo con un 100% artificial.```